In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/salaries.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,EN,CT,AI Research Scientist,80000,EUR,88888,DE,0,DE,L
1,2024,EX,FT,Data Science Director,96000,EUR,106666,ES,100,ES,L
2,2024,EX,FT,Data Science Director,96000,EUR,106666,ES,100,ES,L
3,2024,SE,FT,Data Engineer,130500,USD,130500,US,0,US,M
4,2024,SE,FT,Data Engineer,96000,USD,96000,US,0,US,M


In [ ]:
#Drop the non-benificial columns
df = df.drop(columns=['salary', 'salary_currency', 'company_location'])


In [ ]:
#Drop the years we will not be using
df = df[df['work_year'] > 2022]

In [ ]:
#Drop outlier row on salary > 500000 and experience = EN
df = df.drop(df[(df['salary_in_usd'] > 500000)].index)
#df = df.drop(df[(df['salary_in_usd'] > 500000) & (df['experience_level'] == 'EN')].index)

In [ ]:
#Drop outlier row on salary < 40000 and employee_residence != US
df = df.drop(df[(df['salary_in_usd'] < 40000)].index)
#df = df.drop(df[(df['salary_in_usd'] < 40000) & (df['employee_residence'] != 'US')].index)

In [ ]:
df_sort = df.sort_values(by='salary_in_usd', ascending= False)
df_sort.head(25)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_size
1165,2024,SE,FT,Research Scientist,500000,US,100,M
1255,2024,SE,FT,Research Scientist,484000,US,100,M
4015,2023,EX,FT,Data Engineer,465000,US,0,M
2567,2024,SE,FT,Research Engineer,450000,US,0,M
1109,2024,EX,FT,Head of Machine Learning,448000,US,100,M
2587,2024,SE,FT,Research Scientist,440000,US,0,M
589,2024,MI,FT,ML Engineer,440000,US,0,M
258,2024,MI,FT,Research Engineer,440000,US,0,M
2543,2024,MI,FT,Research Engineer,440000,US,0,M
1067,2024,MI,FT,Research Engineer,440000,US,0,M


In [ ]:
# Determine the number of unique values in each column.
unique = df.nunique()
print(unique)

work_year                2
experience_level         4
employment_type          4
job_title              128
salary_in_usd         2083
employee_residence      60
remote_ratio             3
company_size             3
dtype: int64


In [ ]:
#Look at remote_ratio value counts for binning
remote_ratio_counts = df['remote_ratio'].value_counts()
et_top_10 = remote_ratio_counts.head()
print(et_top_10)

0      7984
100    3341
50       60
Name: remote_ratio, dtype: int64


In [ ]:
#Update 50% remote to 0 implying a physical office for those workers
df.loc[df['remote_ratio'] == 50, 'remote_ratio'] = 0
#Check to make sure binning was sucessful
df['remote_ratio'].value_counts()

0      8044
100    3341
Name: remote_ratio, dtype: int64

In [ ]:
#Look at job_title value counts for binning
job_title_counts = df['job_title'].value_counts()
top_15 = job_title_counts.head(15)
print(top_15)

Data Engineer                     2385
Data Scientist                    2275
Data Analyst                      1673
Machine Learning Engineer         1275
Research Scientist                 393
Applied Scientist                  340
Analytics Engineer                 307
Data Architect                     281
Research Engineer                  232
Business Intelligence Engineer     213
Data Manager                       163
Data Science                       154
ML Engineer                        129
Business Intelligence Analyst      115
Machine Learning Scientist          87
Name: job_title, dtype: int64


In [ ]:
#Apply cutoff to job_titles to be replaced with other
cutoff = 500
job_titles_to_replace = job_title_counts[job_title_counts < cutoff].index.tolist()

#Replace in dataframe
for job in job_titles_to_replace:
  df['job_title'] = df['job_title'].replace(job, "Other")

#Check to make sure binning was sucessful
df['job_title'].value_counts()

Other                        3777
Data Engineer                2385
Data Scientist               2275
Data Analyst                 1673
Machine Learning Engineer    1275
Name: job_title, dtype: int64

In [ ]:
#Look at employee_residence value counts for binning
employee_residence_counts = df['employee_residence'].value_counts()
er_top_10 = employee_residence_counts.head(10)
print(er_top_10)

US    10255
GB      466
CA      288
ES       58
DE       48
AU       30
FR       25
CO       14
IN       14
PT       11
Name: employee_residence, dtype: int64


In [ ]:
#Apply cutoff to employee_residence to be replaced with other
cutoff = 500
employee_residence_to_replace = employee_residence_counts[employee_residence_counts < cutoff].index.tolist()

#Replace in dataframe
for x in employee_residence_to_replace:
  df['employee_residence'] = df['employee_residence'].replace(x , "Other")

#Check to make sure binning was sucessful
df['employee_residence'].value_counts()

US       10255
Other     1130
Name: employee_residence, dtype: int64

In [ ]:
# Binning salary_in_usd for processing
num_bins = 5
min_salary = df['salary_in_usd'].min()
max_salary = df['salary_in_usd'].max()
bin_width = (max_salary - min_salary) / num_bins
bin_edges = [min_salary + i * bin_width for i in range(num_bins + 1)]

# Create a new column to store the bin labels
df['salary_binned'] = pd.cut(df['salary_in_usd'], bins=bin_edges, labels=range(num_bins))

# Drop rows with NaN values in the target variable 'salary_bin'
df = df.dropna(subset=['salary_binned'])

# Convert 'salary_binned' column from category to int64
df['salary_binned'] = df['salary_binned'].astype('int64')

<ipython-input-37-b3aa6f8a6b0d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_binned'] = df['salary_binned'].astype('int64')


In [ ]:
# Display the bins
print(df['salary_binned'].value_counts())


1    5229
0    4601
2    1360
3     162
4      25
Name: salary_binned, dtype: int64


In [ ]:
print("Bin Ranges:")
for i in range(len(bin_edges) - 1):
    print(f"Bin {i+1}: {bin_edges[i]} - {bin_edges[i+1]}")

Bin Ranges:
Bin 1: 40000.0 - 132000.0
Bin 2: 132000.0 - 224000.0
Bin 3: 224000.0 - 316000.0
Bin 4: 316000.0 - 408000.0
Bin 5: 408000.0 - 500000.0


In [ ]:
#Look at company_size value counts for binning
company_size_counts = df['company_size'].value_counts()
cs_top_10 = company_size_counts.head()
print(cs_top_10)


M    10771
L      559
S       47
Name: company_size, dtype: int64


In [ ]:
#export dataFrame to CSV for tableau processing
df.to_csv('/content/drive/MyDrive/Colab Notebooks/processed_salaries.csv', index=False)